In [1]:
# [x] see if we have multi query capabilities
# [x] expose connection radius infrastructure
# what we want:
# [x] we want to create new states first with a specific connection radius - implement a ConnectionFilter instead of trying to import BoundedConnectionStrategy
# [x] then we want to reconfigure the planner to only create new states when adding start and goals without growing the roadmap more (just 'using') - clear input query every time
# [x] decide on experiment apparatus
# Run from d: 2....20
# Naive: Pre-constructed instance (empty, narrow corridor, etc.). Then run with m (~500>) different roadmaps. Each roadmap is a single Monte-Carlo sample.
# Naive: Run n trials on one roadmap <-- problem: number of trials we need to run scales alongside epsilon nets. ironic! ~either sample new states or just reuse graph states? sample lots of states and reuse?
# [x] write up experiment apparatus
# [x] switch to manually defining problems instead of simple setup since we need more control
# [x] define validation of ONE d-dimensional roadmap
# [x] add in multiple d-dimensional roadmaps
# [x] add in control of graph growth in solution criterion
# [x] add in parameter/n prob solution infrastructure
# AS SAMPLES INCREASE m inc.
# maintain a list of tolerances
# [x] add in pandas dataframe charting infrastructure
# current array format: N_trial x M_sample x T_tol -> # frac paths rel opt

# [x] implement StateValidityChecker for d-dimensional snake environments
# [ ] implement a ValidStateSampler so we do not need to rejection-sample in high-dimensional environments
# [ ] work out math and implement the strategic spots we need to check for full environment coverage in snake
# [ ] save more data than just estimated success rate

# [ ] modularize the pieces of the script to be easily integrated into a supercloud mapreduce pattern
# [ ] design parameters and run experiment
# --what is likely going to happen is that given the current level of control we have form the python bindings, since we can only query individual paths, it's going to be too slow.
# --run anyway so we can present data to nick
# --but for interpretability, _now_ formally prove the result that means `if we want uniform converage with \epsilon-optimality' => we need an \epsilon'-net

In [34]:
%env OMPL_PATH= /home/seiji/Research/ompl/py-bindings
from nonasymptotic import ompl
from ompl import base as ob
from ompl import geometric as og

import numpy as np
import pandas as pd
from sympy.combinatorics.graycode import GrayCode
import networkx as nx
from datetime import datetime
import json
import copy
import os

env: OMPL_PATH=/home/seiji/Research/ompl/py-bindings


In [119]:
class GrayCodePathWalls:
    def __init__(self, dim, length, thickness=0.0):
        assert dim >= 2
        assert length > 0

        gray_code = GrayCode(dim - 1)
        gray_coords = []
        for gray_str in gray_code.generate_gray():
            gray_coords.append(tuple(list(map(lambda s: int(s), [*gray_str]))))

        gray_coords_reversed = copy.deepcopy(gray_coords)
        gray_coords_reversed.reverse()

        # linear graph, represented as a list (NOTE: use NetworkX if we need something more powerful) 
        no_walls_linear_list = []
        for i in range(length):
            if i % 2 == 0:
                no_walls_linear_list += self._prepend_coord(i, gray_coords)
            else:
                no_walls_linear_list += self._prepend_coord(i, gray_coords_reversed)

        no_walls_edge_list = list(zip(no_walls_linear_list, no_walls_linear_list[1:]))

        self.no_walls_graph = nx.Graph()
        self.no_walls_graph.add_nodes_from(no_walls_linear_list)
        self.no_walls_graph.add_edges_from(
            no_walls_edge_list)  # we could store the complement, (edge = wall) but that's larger memory

        self.dim = dim
        self.length = length
        self.thickness = thickness

    def distance_to_wall(self, x):
        assert x.shape == (self.dim,)

        # begin by rounding to know which hypercube we are in 
        cube_coords = np.round(x).astype('int64')

        # translate to cube center
        leveler = np.zeros(self.dim)
        leveler[0] += int(x[0])
        x_c = (x - leveler) - ((cube_coords - leveler) / 2 + 0.25)

        # get neighbors so we know there are free passageways
        neighbors = list(self.no_walls_graph.neighbors(tuple(cube_coords)))

        # fill in coordinates of walls, accounting for neighbors with no walls
        walls_low = np.ones(self.dim) * -0.25
        walls_high = np.ones(self.dim) * 0.25

        for i in range(len(neighbors)):
            walls_low, walls_high = self._unblock_wall(
                cube_coords, np.array(neighbors[i]), walls_low, walls_high)

        walls_low_dists = np.abs(x_c - walls_low)
        walls_high_dists = np.abs(x_c - walls_high)

        return min(np.min(walls_low_dists), np.min(walls_high_dists)) - self.thickness

    @staticmethod
    def _unblock_wall(cube_coords, neighbor_coords, walls_low, walls_high):
        neighbor_diff = neighbor_coords - cube_coords
        neighbor_diff_ind = np.where(neighbor_diff)
        neighbor_diff_sign = neighbor_diff[neighbor_diff_ind]

        if neighbor_diff_sign > 0:
            walls_high[neighbor_diff_ind] = np.inf
        else:
            walls_low[neighbor_diff_ind] = -np.inf

        return walls_low, walls_high

    @staticmethod
    def _prepend_coord(coord, list_of_coords):
        return list(
            map(
                lambda tup: (coord,) + tup,
                list_of_coords
            )
        )



In [138]:
env = GrayCodePathWalls(2, 5)
print(env.distance_to_wall(np.array([0.25, 1.0])))

[0. 0.]
[0.   0.25]
0.0


In [2]:
# defining free parameters: dimension, obstacles (i.e. state validity)
sample_schedule = [10, 100, 1000]  #[10, 100, 1000, 10000, 100000]
tol = 0.5
delta = 0.25
n_trials = 10
ds = [2, 3]  #[2, 6, 10, 14, 17, 20]
r = tol / np.sqrt(1 + tol * tol) * delta
n_paths = 5  # should be a function of dimensionality in the future for coverage
is_state_valid = lambda state: True

In [3]:
# make a new log directory and save experiment params
name = 'long'
date_str = datetime.now().strftime('%m-%d-%Y')
log_dir = 'data/' + date_str + '_' + name
try:
    os.mkdir(log_dir)
except FileExistsError:
    print('WARNING: Made that log directory already!')

params = {
    "sample_schedule": sample_schedule,
    "tol": tol,
    "ds": ds,
    "n_paths": n_paths,
    "n_trials": n_trials,
}

with open(os.path.join(log_dir, 'params.json'), 'w') as handle:
    json.dump(params, handle, indent=4)


In [ ]:
for d in ds:
    # set up unit cube space with obstacles
    space = ob.RealVectorStateSpace(d)  # TODO: rather than construct a unit cube, construct the snake environment
    space.setBounds(0.0, 1.0)  # we'll need to do that with a new type of state validity checker

    si = ob.SpaceInformation(space)
    si.setStateValidityChecker(ob.StateValidityCheckerFn(is_state_valid))
    pdef = ob.ProblemDefinition(si)

    # TODO: define a ValidStateSampler to sample directly from the space without having to evaluate isValid

    # set up the PRM planner
    planner = og.PRM(si)
    accept_if_within_r = lambda v1, v2: planner.distanceFunction(v1, v2) <= r
    planner.setConnectionFilter(og.ConnectionFilter(accept_if_within_r))
    planner.setProblemDefinition(pdef)
    planner.setup()  # no harm in recalling if setup already

    rec = np.zeros((n_trials, len(sample_schedule)))

    for i_trial in range(n_trials):
        planner.clear()
        pdef.clearStartStates()
        pdef.clearGoal()

        # save fraction of rel-optness
        for i_sample, m_samples in enumerate(sample_schedule):
            num_rel_opt_paths = 0
            graph_has_m_samples = lambda: planner.milestoneCount() >= m_samples
            term_cond = ob.PlannerTerminationConditionFn(graph_has_m_samples)
            for i_path in range(n_paths):
                # set up problem instance
                start = ob.State(space)
                start.random()

                goal = ob.State(space)
                goal.random()  # TODO: based on accuracy tolerance, choose starts and goals strategically based on space construction

                # compute optimum (remember to turn objects into pointers!)
                opt_len = space.distance(start(), goal())

                planner.clearQuery()  # clear previous queries so we can multi-query
                pdef.setStartAndGoalStates(start, goal)

                # need to grow first to get to the # of states we want
                planner.growRoadmap(term_cond)

                # allow to create _one_ state
                solved = planner.solve(0.001)

                if str(solved) == 'Exact solution':
                    path = pdef.getSolutionPath()
                    length = path.length()
                    opt_tol = (1 + tol) * opt_len
                    within_tols = int(length < opt_tol)
                    num_rel_opt_paths += within_tols

            rec[i_trial, i_sample] = float(
                num_rel_opt_paths) / n_paths  # TODO: save more data than just an estimated success rate

    # save data
    df = pd.DataFrame(rec, index=range(n_trials), columns=sample_schedule)
    df.to_pickle(os.path.join(log_dir, 'prm_%s_d%i.pkl' % (date_str, d)))
